In [67]:
from datetime import datetime, timedelta
import pytz
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
pd.options.mode.chained_assignment = None
from statistics import stdev
import pandas as pd
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [68]:
market = Market()
sec = SEC()
fin = ADatabase("ratios")
umod = UniversalModeler()

In [4]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [56]:
factors = [
"current_ratio",
    "longterm_debt_to_equity",
    "debt_to_equity",
    "debt_to_assets",
"cash_return_on_assets",
          "price_to_earnings"]

In [57]:
included_columns = ["year","quarter","ticker","adjclose","y"]

In [58]:
included_columns.extend(factors)

In [59]:
tax_rate_col = "effectiveincometaxratereconciliationatfederalstatutoryincometaxrate"
capex_col = "capitalexpendituresincurredbutnotyetpaid"
cfo_col = "netcashprovidedbyusedinoperatingactivities"

In [60]:
training_sets = []
sec.connect()
market.connect()
column_database = {}
for i in tqdm(range(sp5.index.size)):
    try:
        row = sp5.iloc[i]
        ticker = row["Symbol"]
        cik = int(row["CIK"])
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices = prices.groupby(["year","quarter","ticker"]).mean().reset_index()
        financials = sec.retrieve_filing_data(int(cik))
        financials = p.column_date_processing(financials)
        financials["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in financials["filed"]]
        financials["year"] = [x.year for x in financials["date"]]
        financials["quarter"] = [x.quarter for x in financials["date"]]
        financials = financials[financials["form"]=="10-K"]
        financials["current_ratio"] = financials["assetscurrent"] / financials["liabilitiescurrent"]
        financials["longterm_debt_to_equity"] = financials["longtermdebt"] / financials["stockholdersequity"]
        financials["debt_to_equity"] = (financials["longtermdebt"] + financials["debtcurrent"]) / financials["stockholdersequity"]
        financials["debt_to_assets"] = financials["longtermdebt"] / financials["assets"]
        financials["cash_return_on_assets"] = financials[cfo_col] / financials["assets"]
        ticker_data = prices.copy()
        ticker_data = ticker_data.merge(financials,on=["year","quarter"],how="left").reset_index()
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","quarter"]).mean().reset_index()
        ticker_data["ticker"] = ticker
        ticker_data["price_to_earnings"] = ticker_data["adjclose"] / ticker_data["earningspersharebasic"]
        ticker_data["y"] = ticker_data["adjclose"].shift(-4)
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except Exception as e:
        print(ticker,str(e))
        continue
sec.disconnect()
market.disconnect()

  0%|▋                                                                                                                                                                        | 2/490 [00:00<02:44,  2.97it/s]

AOS 'debtcurrent'


  1%|█                                                                                                                                                                        | 3/490 [00:00<02:22,  3.41it/s]

ABT 'debtcurrent'


  1%|█▍                                                                                                                                                                       | 4/490 [00:01<02:09,  3.76it/s]

...

ZBH 'debtcurrent'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 489/490 [02:55<00:00,  1.89it/s]

ZION 'assetscurrent'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [02:56<00:00,  2.78it/s]

ZTS 'longtermdebt'


In [61]:
data = pd.concat(training_sets)

In [62]:
data = data.dropna().sort_values("year")

In [70]:
data

,year,quarter,ticker,adjclose,y,current_ratio,longterm_debt_to_equity,debt_to_equity,debt_to_assets,cash_return_on_assets,price_to_earnings
72,2018,1,MMM,197.909534,172.739779,1.869912,1.128768,1.257994,0.356672,0.186189,53.502209
76,2019,1,MMM,172.739779,139.688633,1.874355,1.280678,1.424130,0.382064,0.180096,44.385367
80,2020,1,MMM,139.688633,164.946260,1.620309,1.687648,1.889370,0.421940,0.165755,36.437632
84,2021,1,MMM,164.946260,151.163195,1.628014,1.663410,1.820454,0.414573,0.156676,41.633980
88,2022,1,MMM,151.163195,111.998582,1.789142,1.294379,1.370078,0.382668,0.159839,16.496529
...,...,...,...,...,...,...,...,...,...,...,...
84,2021,1,WAT,273.060656,325.357903,1.943344,1.425237,1.556425,0.503986,0.251751,71.329072
88,2022,1,WAT,325.357903,323.464355,2.039713,3.116870,3.270663,0.512228,0.244979,74.452609
4,2017,1,WTW,116.257023,145.625898,1.004738,0.804132,0.925817,0.201377,0.022271,56.735903
8,2018,1,WTW,145.625898,156.468370,1.054329,0.558467,0.587836,0.179809,0.021676,79.854680


In [69]:
fin.connect()
years = 4
for modeler in [umod]:
    for year in tqdm(range(2016,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.model(training_slice,prediction_set,factors,tf=False)
        stuff["training_years"] = years
        fin.store(set_name,stuff)
fin.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:27<00:00,  3.92s/it]
